In [1]:
# !pip install transformers
# !wget https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.csv
# Put me under data/

In [2]:
# import btorch

In [3]:
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
# from tqdm.notebook import tqdm
import pandas as pd
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoModelForSequenceClassification,AutoModel
from transformers import AutoTokenizer, AutoConfig
from sklearn.utils import shuffle
import sklearn
import random

seed = 888
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

Using cuda:1


In [4]:
# path of data and the name of pretrained weights
# path = '../input/nlpdata/labeled_data.csv'
path = './data/labeled_data.csv'

In [5]:
df = pd.read_csv(path, index_col = 0)
df = shuffle(df)
df.describe()

,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [6]:
# 364 & 11176, Just to make sure the experiment is reproducible
df.iloc[0:2] 

,count,hate_speech,offensive_language,neither,class,tweet
364,3,1,2,0,1,"""@likemicah: Y'all dumb ass niggas be so press..."
11176,3,0,3,0,1,I snorted shit with more girls in my presence ...


In [7]:
df['class'].to_numpy()

array([1, 1, 1, ..., 1, 2, 2], dtype=int64)

In [8]:
# btorch.utils.get_class_weight(df['class'].to_numpy(), 'sklearn')

In [9]:
# split data into train and test
train_data = df.sample(frac = 0.8)
valid_data = df.drop(train_data.index)
test_data = valid_data.sample(frac = 0.5)
valid_data = valid_data.drop(test_data.index)

display(train_data.head())
print("===================================")
display(test_data.head())
print(train_data.shape)
print(test_data.shape)

,count,hate_speech,offensive_language,neither,class,tweet
5278,3,0,0,3,2,@WebAntOnYT btw Jim crow laws were supported b...
8208,3,0,3,0,1,Before I let a pussy cut my hair I just will l...
16962,3,0,0,3,2,RT @NASAWatch: .@tweetsoutloud we found your b...
20491,3,0,2,1,1,RT @sorryimalex: idkmadz tbh I've seen you in ...
4523,3,0,3,0,1,"@PaulConroy @habeshasuperman oh, it's coming u..."


,count,hate_speech,offensive_language,neither,class,tweet
4883,3,1,0,2,2,"@StarbucksSanae Are you sure about that, slant..."
2946,3,0,3,0,1,@Coley_Cee bitch dats yo sahara desert ass
11082,3,1,2,0,1,I never seen a Asian or Chinese bitch pregnant...
22484,3,0,3,0,1,"This bitch wanna smoke all of my weed,"
1305,3,0,0,3,2,&#8220;@GianneNichole: Lakers really trash.&#8...


(19826, 6)
(2478, 6)


# Build Dataset

The data are stored as a CSV and as a pickled pandas dataframe (Python 2.7). Each data df contains 5 columns:

count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

hate_speech = number of CF users who judged the tweet to be hate speech.

offensive_language = number of CF users who judged the tweet to be offensive.

neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, model_name=None,  train = True, device='cuda'):
        super(Dataset, self).__init__()
        self.df = df
        self.device = device
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        device = self.device
        count = torch.LongTensor([self.df.iloc[idx]['count']])
        hate_speech = torch.LongTensor([self.df.iloc[idx]['hate_speech']])
        offensive_language = torch.LongTensor([self.df.iloc[idx]['offensive_language']])
        neither = torch.LongTensor([self.df.iloc[idx]['neither']])
        target = torch.LongTensor([self.df.iloc[idx]['class']])
        tweet = self.df.iloc[idx]['tweet']
        return (count.to(device), hate_speech.to(device), offensive_language.to(device)
                , neither.to(device), target.to(device), tweet)

In [11]:
dataset = Dataset(df, device=device)
dataloader = torch.utils.data.DataLoader(dataset, batch_size= 6, shuffle=True)
output = next(iter(dataloader))
print(output)

[tensor([[3],
        [3],
        [3],
        [3],
        [3],
        [3]], device='cuda:1'), tensor([[0],
        [3],
        [0],
        [0],
        [0],
        [0]], device='cuda:1'), tensor([[3],
        [0],
        [3],
        [0],
        [3],
        [0]], device='cuda:1'), tensor([[0],
        [0],
        [0],
        [3],
        [0],
        [3]], device='cuda:1'), tensor([[1],
        [0],
        [1],
        [2],
        [1],
        [2]], device='cuda:1'), ('RT @BCniggas: I know plenty of hoes in love with they niggas but it be the same bitch fuckin niggas', 'fuck ray rice ,someone crack that bitch', 'Yo bitch she boof she got that throwbac pussy !', "RT @UFlorida: @GatorZoneFB It's a bird...It's a plane...It's Jake McGee! http://t.co/WVdLScoDvw", 'RT @CourtneyyKay: Cruising down the street in my 64, jockin the bitches, slappin the hoes', 'Early bird gets the worm! &#128027;&#128036;')]


# Utils

In [12]:
def number_params(model, exclude_freeze=False):
    """calculate the number of parameters in a model

    Args:
        model (nn.Module): PyTorch model
        exclude_freeze (bool, optional): Whether to count the frozen layer. Defaults to False.
    """
    pp = 0
    for p in list(model.parameters()):
        if exclude_freeze and p.requires_grad is False:
            continue
        nn = 1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

def finetune(
        model: nn.Module,
        base_lr: float,
        groups,
        ignore_the_rest: bool = False,
        raw_query: bool = False,
        regex=False):
    """ This is something call per-parameter options

    Separate out the finetune parameters with a learning rate for each layers of parameters
    This function only support setting a different learning rate for each layer's arameter.
    Depending on the optimizer, you can set extra parameter for that layer for the optmizer -> See Notes 
    If you freeze layer using this function and want to unfreeze it later:
    See https://discuss.pytorch.org/t/correct-way-to-freeze-layers/26714/2

    Args:
        model (nn.Module): Pytorch Model
        base_lr (float): learning rate of all layers
        groups (Dict[str, float]): key is `name` of layers, value is the `extra_lr` (or False).
          all layers that contains that `name` will have `lr` of base_lr*extra_lr.
          it uses fnmatch|regex to check whether a layer contains that `name`.
          fnmatch is matching structure like `layer1*`, `layer?.conv?.`, `*conv2*`, etc...
          regex is the comman regex matching.
          Hence, `name` here is either fnmatch or regex expression if using raw_query.
          If `float` is False: those layers with `name` will be freeze. 
          In particular, they will not be included in the return output and require_grad will be set to False
        ignore_the_rest (bool, optional): Include the remaining layer that are not stated in `grouprs` or not. Defaults to False.
        raw_query (bool, optional): Modify the keys of `groups` as f'*{key}*' if False. Only useful when `regex=False`
          Do not do any modification to the keys of `groups` if True. Defaults to False.
        regex (bool, optional): Use regex instead of fnmatch on keys of groups. Defaults to False.
          This will overrride raw_query to True. 
          Notice: `regex=False` is depracted

    Returns:
        List[Dict[str, Union[float, Iterable]]]: list of dict that has two or more key-value pair.
          The first one is feature generation layers. [those layers must start with `features` name] <usually is backbone>
            is a dict['params':list(model.parameters()), 'names':list(`layer's name`), 'query':query, 'lr':base_lr*groups[groups.keys()]]
          The remaining are all others layer. [all others params for last one, if ignore_the_rest = False]
            is a dict['params':list(model.parameters()), 'names':list(`layer's name`), 'lr':base_lr]

    Examples:
        >>> model = models.resnet50()
        >>> # all layers that has name start with `layer1 and layer2` will have learning rate `0.001*0.01`
        >>> # all layers that has name start with `layer3` will be froozen`
        >>> # all layers that has name start with `layer4` will have learning rate `0.001*0.001`
        >>> # for all other layers will have the base_lr `0.001`
        >>> model_params = finetune(model, base_lr=0.001, groups={'^layer[1-2].*': 0.01, '^layer3.*': False, '^layer4.*': 0.001}, regex=True)
        >>> # setting extra parameter (other than learning rate) for that optimizer
        >>> # the second param_group `layer4` will have weight_decay 1e-2
        >>> model_params[1]['weight_decay'] = 1e-2
        >>> # init optimizer with the above setting
        >>> # the argument under `torch.optim.SGD` will be overrided by finetune() if they exist.
        >>> # For example, all model_params will have weight_decay=5e-3 except model_params[1]
        >>> optimizer = torch.optim.SGD(model_params, momentum=0.9, lr=0.1, weight_decay=5e-3)
    """
    if regex:
        raw_query = True
    else:
        warnings.warn("regex=False is deprecated; use regex=True", DeprecationWarning)
    # Deal with Freeze Later
    freeze_group = dict()
    freeze = False
    for k,v in groups.items():
        if v is False:
            freeze_group[k] = 1
            freeze=True
    for k in freeze_group.keys():
        del groups[k]
    freeze_group = "(" + ")|(".join(freeze_group) + ")"

    parameters = [
        dict(params=[],
             names=[],
             query=query if raw_query else '*' + query + '*',
             lr = lr * base_lr,
             initial_lr = lr * base_lr) for query, lr in groups.items()
    ]
    rest_parameters = dict(params=[], names=[], lr=base_lr, initial_lr=base_lr)
    for k, v in model.named_parameters():
        rest = 0
        if freeze and regex and re.match(freeze_group, k):
            v.requires_grad = False
            continue
        for group in parameters:
            if not regex and fnmatch(k, group['query']):
                group['params'].append(v)
                group['names'].append(k)
                rest = 1
                break
            elif regex and re.compile(group['query']).search(k):
                group['params'].append(v)
                group['names'].append(k)
                rest = 1
                break
        if rest == 0:
            rest_parameters['params'].append(v)
            rest_parameters['names'].append(k)

    if not ignore_the_rest:
        parameters.append(rest_parameters)
    for group in parameters:
        group['params'] = iter(group['params'])
    return parameters

# Model

In [13]:
class LanguageModel(nn.Module):
    
    def __init__(self, model_name, num_labels = 3, freeze_pretrained=False):
        super(LanguageModel, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if freeze_pretrained:
            print("You are freezing the BERT")
            for name, p in self.model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
        print(f"Total number of params: {number_params(self.model)}")
        print(f"Total number of trainable params: {number_params(self.model, exclude_freeze=True)}")

    def forward(self, src, has_mask=False):
        # print(src)
        output = self.tokenizer(src, padding=True, truncation=True, max_length=50)
        output = torch.LongTensor(output['input_ids']).to(device)
        if has_mask == True:
            attention_mask=(output != 0).float() # here `0` is the <pad> token, i guess
            output = self.model(output, attention_mask=attention_mask)
        else:
            output = self.model(output)
        return output

In [14]:
class LastAttnModel(nn.Module):
    """
    Use the [CLS] as query and all other output as key and values.
    Pass it to a Multi-Head Attention, then a Linear classifier
    
    Args:
      auxiliary_head(list(int)): Only used when training
        - list of idx of hidden_layers that will be used as auxiliary_head. Here `idx` start from 1
        - See BertConfig['num_hidden_layers'] for total number of layers
        - EG: `auxiliary_head=[10,11,12]`.
      last_hidden_layer(int): Treat the output of this layer as last_hidden_layer

    Examples:
      tweet = iter(dataloader_train).next()[-1]
      modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
      out, attn_weight = modelA(list(tweet))
      print(out.shape, attn_weight.shape) #torch.Size([32, 3]) torch.Size([32, 1, 49])
      
    Returns:
      list(tensor): the first tensor is the prediction, the second is the attention weight
    """
    
    def __init__(self, pretrain_model, tokenizer, 
                 last_attn_num_head = 8,
                 classifier_hidden_dim = 512, 
                 classifier_dropout = 0,
                 num_labels = 3, 
                 freeze_pretrained=False,
                 auxiliary_head=None,
                 last_hidden_layer=-1):
        super(LastAttnModel, self).__init__()
        self.pretrain_model = pretrain_model
        self.tokenizer = tokenizer
        self.auxiliary_head = auxiliary_head
        self.num_layers = len(pretrain_model.encoder.layer)
        self.last_hidden_layer = last_hidden_layer
        
        if freeze_pretrained:
            if self.auxiliary_head is not None:
              warnings.warn("freeze_pretrained and auxiliary_head set to True together is useless for training. Consider use `finetune()`")
            print("You are freezing the BERT pertrain")
            for name, p in self.pretrain_model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
        
        embed_size = pretrain_model.embeddings.word_embeddings.embedding_dim
        self.last_attn = nn.MultiheadAttention(embed_size, last_attn_num_head, batch_first=True)
        self.final_classifier = nn.Sequential(
            nn.Linear(embed_size, classifier_hidden_dim),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(classifier_hidden_dim, num_labels)
        )

        # deal with aux 
        if self.auxiliary_head is not None:
          self.aux_classifiers = nn.ModuleList()
          for i in self.auxiliary_head:
            self.aux_classifiers.append(nn.Sequential(
              nn.Linear(embed_size, classifier_hidden_dim),
              nn.ReLU(),
              nn.Dropout(classifier_dropout),
              nn.Linear(classifier_hidden_dim, num_labels)
            ))


        print(f"Total number of params: {number_params(self)}")
        print(f"Total number of trainable params: {number_params(self, exclude_freeze=True)}")
    def forward(self, src, has_mask=False):
        # print(src)
        out = []
        tokens = self.tokenizer(src, padding=True, truncation=True, max_length=50)
        inputs = torch.LongTensor(tokens['input_ids']).to(device)
        if has_mask == True:
            # attention_mask=(inputs != 0).float() # here `0` is the <pad> token, i guess
            attention_mask = torch.LongTensor(tokens['attention_mask']).to(device)
            pre_train_output = self.pretrain_model(inputs, attention_mask=attention_mask)
        else:
            pre_train_output = self.pretrain_model(inputs)
        # last_hidden = pre_train_output["last_hidden_state"]      # (N,T,E)
        last_hidden = pre_train_output["hidden_states"][self.last_hidden_layer]
        last_hidden_state_cls = last_hidden[:, 0, :].unsqueeze(1)   # (N,1,E)
        last_hidden_state_rest = last_hidden[:, 1:, :] # (N,T-1,E)
        atten_mask_pad = (inputs == 0)[:,1:] #(N,T-1)
        last_attn_out, last_attn_w = self.last_attn(last_hidden_state_cls, last_hidden_state_rest, last_hidden_state_rest,
                                                    key_padding_mask=atten_mask_pad) #(N,1,E), (N,1,T-1)
        last_attn_out = last_attn_out.squeeze(1) #(N,E)
        output = self.final_classifier(last_attn_out)
        out += [output, last_attn_w]
        ## auxiliary_head
        if self.auxiliary_head is not None:
          if "hidden_states" not in pre_train_output:
            raise Exception("Put `pre_train_output=True` in AutoConfig")
          for idx in range(len(self.aux_classifiers)):
            hidden_cls = pre_train_output["hidden_states"][self.auxiliary_head[idx-1]][:, 0, :] # (N,E)
            out.append(self.aux_classifiers[idx](hidden_cls))
        return out
    
# ## Usage
# tweet = iter(dataloader_train).next()[-1]
# modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
# out, attn_weight = modelA(list(tweet))
# print(out.shape, attn_weight.shape)

# Initialization

In [15]:
dataset1 = Dataset(train_data)
dataloader_train1 = torch.utils.data.DataLoader(dataset1, batch_size=32, shuffle=True)
dataset_valid1 = Dataset(valid_data)
dataloader_valid1 = torch.utils.data.DataLoader(dataset_valid1, batch_size=32, shuffle=False)
dataset_test1 = Dataset(test_data)
dataloader_test1 = torch.utils.data.DataLoader(dataset_test1, batch_size=32, shuffle=False)

In [16]:
lossfn1_1 = nn.CrossEntropyLoss().to(device)
# lossfn = nn.CrossEntropyLoss(weight=torch.tensor([5.7769, 0.4305, 1.9844])).to(device)
# model = LanguageModel(model_name, freeze_pretrained=False).to(device)
model_name1_1 = 'bert-base-uncased'
config1_1 = AutoConfig.from_pretrained(
    model_name1_1, 
    num_labels = 2,
    output_hidden_states = True,
    output_attention = False,
    hidden_dropout_prob = 0.2,
) 
print(config1_1)
pretrain_model1_1 = AutoModel.from_pretrained(
    model_name1_1,
    config = config1_1
).to(device)

tokenizer1_1 = AutoTokenizer.from_pretrained(model_name1_1)
model1_1 = LastAttnModel(pretrain_model1_1, tokenizer1_1, freeze_pretrained=True, num_labels = 2,
                      classifier_dropout=0.2,
                      auxiliary_head=None,
                      last_hidden_layer=5).to(device)
# model_params = finetune(model, base_lr=1e-4, 
#                         groups={'^pretrain_model.encoder.layer.([0-7])\..*': False, 
#                                 '^pretrain_model.encoder.layer.([8-9]|1[012]).*': 0.01, 
#                                 '^pretrain_model.pooler.*': False,
#                                 '^pretrain_model.embeddings.*':False},
#                         regex=True)
optimizer1_1 = torch.optim.Adam(model1_1.parameters(), lr=1e-4, weight_decay=1e-3)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


You are freezing the BERT pertrain
Total number of params: 112239362
Total number of trainable params: 2757122


In [17]:
lossfn1_2 = nn.CrossEntropyLoss().to(device)
# lossfn = nn.CrossEntropyLoss(weight=torch.tensor([5.7769, 0.4305, 1.9844])).to(device)
# model = LanguageModel(model_name, freeze_pretrained=False).to(device)
model_name1_2 = 'bert-base-uncased'
config1_2 = AutoConfig.from_pretrained(
    model_name1_2, 
    num_labels = 2,
    output_hidden_states = True,
    output_attention = False,
    hidden_dropout_prob = 0.2,
) 
print(config1_2)
pretrain_model1_2 = AutoModel.from_pretrained(
    model_name1_2,
    config = config1_2
).to(device)

tokenizer1_2 = AutoTokenizer.from_pretrained(model_name1_2)
model1_2 = LastAttnModel(pretrain_model1_2, tokenizer1_2, freeze_pretrained=True, num_labels = 2,
                      classifier_dropout=0.2,
                      auxiliary_head=None,
                      last_hidden_layer=5).to(device)
# model_params = finetune(model, base_lr=1e-4, 
#                         groups={'^pretrain_model.encoder.layer.([0-7])\..*': False, 
#                                 '^pretrain_model.encoder.layer.([8-9]|1[012]).*': 0.01, 
#                                 '^pretrain_model.pooler.*': False,
#                                 '^pretrain_model.embeddings.*':False},
#                         regex=True)
optimizer1_2 = torch.optim.Adam(model1_2.parameters(), lr=1e-4, weight_decay=1e-3)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


You are freezing the BERT pertrain
Total number of params: 112239362
Total number of trainable params: 2757122


In [18]:
lossfn1_3 = nn.CrossEntropyLoss().to(device)
# lossfn = nn.CrossEntropyLoss(weight=torch.tensor([5.7769, 0.4305, 1.9844])).to(device)
# model = LanguageModel(model_name, freeze_pretrained=False).to(device)
model_name1_3 = 'bert-base-uncased'
config1_3 = AutoConfig.from_pretrained(
    model_name1_3, 
    num_labels = 2,
    output_hidden_states = True,
    output_attention = False,
    hidden_dropout_prob = 0.2,
) 
print(config1_3)
pretrain_model1_3 = AutoModel.from_pretrained(
    model_name1_3,
    config = config1_3
).to(device)

tokenizer1_3 = AutoTokenizer.from_pretrained(model_name1_3)
model1_3 = LastAttnModel(pretrain_model1_3, tokenizer1_3, freeze_pretrained=True, num_labels = 2,
                      classifier_dropout=0.2,
                      auxiliary_head=None,
                      last_hidden_layer=5).to(device)
# model_params = finetune(model, base_lr=1e-4, 
#                         groups={'^pretrain_model.encoder.layer.([0-7])\..*': False, 
#                                 '^pretrain_model.encoder.layer.([8-9]|1[012]).*': 0.01, 
#                                 '^pretrain_model.pooler.*': False,
#                                 '^pretrain_model.embeddings.*':False},
#                         regex=True)
optimizer1_3 = torch.optim.Adam(model1_3.parameters(), lr=1e-4, weight_decay=1e-3)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


You are freezing the BERT pertrain
Total number of params: 112239362
Total number of trainable params: 2757122


In [19]:
lossfns1 = [lossfn1_1, lossfn1_2, lossfn1_3]
models1 = [model1_1, model1_2, model1_3]
optimizers1 = [optimizer1_1, optimizer1_2, optimizer1_3]

## DEBUG

In [20]:
# model_name = 'bert-base-uncased'
# config = AutoConfig.from_pretrained(
#     model_name, 
#     output_hidden_states = True,
#     output_attention = False
# ) 
# print(config)
# model = AutoModel.from_pretrained(
#     model_name,
#     config = config
# ).to(device)

# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [21]:
# tweet = iter(dataloader_train).next()[-1]
# tokens = tokenizer(list(tweet), padding=True, truncation=True, max_length=50)
# print(tokens.keys())
# inputs = torch.LongTensor(tokens['input_ids']).to(device)
# atten_mask = torch.LongTensor(tokens['attention_mask']).to(device)
# print(inputs.shape)
# hiddens = model(inputs)
# print(hiddens.keys())
# print(hiddens['last_hidden_state'].shape)

In [22]:
# (inputs[0]!=0).float()

In [23]:
# model.embeddings.word_embeddings.embedding_dim

In [24]:
# atten_mask_pad = (inputs == 0)
# print(atten_mask_pad.shape)
# print(atten_mask_pad[:,1:].shape)

# Train

In [25]:
def train1(dataloader_train, dataloader_valid = None, dataloader_test = None,
           models1 = None, optimizers1 = None, lossfns1 = None,
           epochs = 10, has_mask = True):
    
    trainloss = []
    validloss = []
    testloss = []
    trainscore = []
    validscore = []
    testscore = []
    
    trainscore_voted = []
    validscore_voted = []
    testscore_voted = []

    for i in range(epochs):
        averageloss = [0, 0, 0]
        averagescore = [0, 0, 0]
        averagescore_voted = 0
        trainrandom = [0, 0, 0]
        for datas in tqdm(dataloader_train):
            count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
            
            targets_new = [
                # 0 = neither hate speech or offensive language
                # 1 = either hate speech or offensive language
                torch.LongTensor([[0] if t == 2 else [1] for t in target]).to(device),

                # 0 = either offensive language or neither
                # 1 = hate speech
                torch.LongTensor([[1] if t == 0 else [0] for t in target]).to(device),

                # 0 = either hate speech or neither
                # 1 = offensive language
                torch.LongTensor([[1] if t == 1 else [0] for t in target]).to(device)
            ]
            
            preds = []
            
            for mi in range(len(models1)):
                models1[mi].train()
                optimizers1[mi].zero_grad()
                pred = models1[mi](list(tweet), has_mask)
                preds.append(pred[0].argmax(-1).cpu().numpy())
                loss = lossfns1[mi](pred[0], targets_new[mi].squeeze(1))
                for j in range(2, len(pred)):
                    loss+=0.3*lossfns1[mi](pred[j], targets_new[mi].squeeze(1))
                loss.backward()
                optimizers1[mi].step()
                
                f1score = sklearn.metrics.f1_score(targets_new[mi].squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                averageloss[mi] += loss.item()/len(dataloader_train)
                averagescore[mi] += f1score/len(dataloader_train)
            
            preds_voted = np.empty(preds[0].shape[0])
            for pi in range(len(preds[0])):
                hatespeech = 0
                offensivelanguage = 0
                neither = 0
                
                # 0 = neither hate speech or offensive language
                # 1 = either hate speech or offensive language
                if preds[0][pi] == 0:
                    neither +=1
                elif preds[0][pi] == 1:
                    hatespeech += 1
                    offensivelanguage += 1
                
                # 0 = either offensive language or neither
                # 1 = hate speech
                if preds[1][pi] == 0:
#                     pass
                    offensivelanguage += 1
                    neither += 1
                elif preds[1][pi] == 1:
                    hatespeech += 1
                
                # 0 = either hate speech or neither
                # 1 = offensive language
                if preds[2][pi] == 0:
#                     pass
                    hatespeech += 1
                    neither += 1
                elif preds[2][pi] == 1:
                    offensivelanguage += 1                
                
                maxvotes = max(hatespeech, offensivelanguage, neither)
                maxvotes_classes_count = 0
                maxvotes_classes_indexes = []
                if hatespeech == maxvotes:
                    maxvotes_classes_count += 1
                    maxvotes_classes_indexes.append(0)
                if offensivelanguage == maxvotes:
                    maxvotes_classes_count += 1
                    maxvotes_classes_indexes.append(1)
                if neither == maxvotes:
                    maxvotes_classes_count += 1
                    maxvotes_classes_indexes.append(2)
                    
                preds_voted[pi] = random.choice(maxvotes_classes_indexes)
                
                if len(maxvotes_classes_indexes) == 1:
                    trainrandom[0] += 1
                elif len(maxvotes_classes_indexes) == 2:
                    trainrandom[1] += 1
                elif len(maxvotes_classes_indexes) == 3:
                    trainrandom[2] += 1
                
            f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), preds_voted, average = 'weighted')
            averagescore_voted += f1score/len(dataloader_train)
            
        trainloss.append(averageloss)
        trainscore.append(averagescore)
        trainscore_voted.append(averagescore_voted)
        
        if dataloader_valid is not None:
            averageloss = [0, 0, 0]
            averagescore = [0, 0, 0]
            averagescore_voted = 0
            validrandom = [0, 0, 0]
            for datas in tqdm(dataloader_valid):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                
                targets_new = [
                    # 0 = neither hate speech or offensive language
                    # 1 = either hate speech or offensive language
                    torch.LongTensor([[0] if t == 2 else [1] for t in target]).to(device),
                    
                    # 0 = either offensive language or neither
                    # 1 = hate speech
                    torch.LongTensor([[1] if t == 0 else [0] for t in target]).to(device),
                    
                    # 0 = either hate speech or neither
                    # 1 = offensive language
                    torch.LongTensor([[1] if t == 1 else [0] for t in target]).to(device)
                ]
                
                preds = []
                
                for mi in range(len(models1)):
                    models1[mi].eval()
                    pred = models1[mi](list(tweet), has_mask)
                    preds.append(pred[0].argmax(-1).cpu().numpy())
                    loss = lossfns1[mi](pred[0], targets_new[mi].squeeze(1))
                    f1score = sklearn.metrics.f1_score(targets_new[mi].squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                    averageloss[mi] += loss.item()/len(dataloader_valid)
                    averagescore[mi] += f1score/len(dataloader_valid)
                    
                preds_voted = np.empty(preds[0].shape[0])
                for pi in range(len(preds[0])):
                    hatespeech = 0
                    offensivelanguage = 0
                    neither = 0

                    # 0 = neither hate speech or offensive language
                    # 1 = either hate speech or offensive language
                    if preds[0][pi] == 0:
                        neither +=1
                    elif preds[0][pi] == 1:
                        hatespeech += 1
                        offensivelanguage += 1

                    # 0 = either offensive language or neither
                    # 1 = hate speech
                    if preds[1][pi] == 0:
#                         pass
                        offensivelanguage += 1
                        neither += 1
                    elif preds[1][pi] == 1:
                        hatespeech += 1

                    # 0 = either hate speech or neither
                    # 1 = offensive language
                    if preds[2][pi] == 0:
#                         pass
                        hatespeech += 1
                        neither += 1
                    elif preds[2][pi] == 1:
                        offensivelanguage += 1                

                    maxvotes = max(hatespeech, offensivelanguage, neither)
                    maxvotes_classes_count = 0
                    maxvotes_classes_indexes = []
                    if hatespeech == maxvotes:
                        maxvotes_classes_count += 1
                        maxvotes_classes_indexes.append(0)
                    if offensivelanguage == maxvotes:
                        maxvotes_classes_count += 1
                        maxvotes_classes_indexes.append(1)
                    if neither == maxvotes:
                        maxvotes_classes_count += 1
                        maxvotes_classes_indexes.append(2)

                    preds_voted[pi] = random.choice(maxvotes_classes_indexes)

                    if len(maxvotes_classes_indexes) == 1:
                        validrandom[0] += 1
                    elif len(maxvotes_classes_indexes) == 2:
                        validrandom[1] += 1
                    elif len(maxvotes_classes_indexes) == 3:
                        validrandom[2] += 1

                f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), preds_voted, average = 'weighted')
                averagescore_voted += f1score/len(dataloader_valid)
                
            validloss.append(averageloss)
            validscore.append(averagescore)
            validscore_voted.append(averagescore_voted)
            
        if dataloader_test is not None:
            averageloss = [0, 0, 0]
            averagescore = [0, 0, 0]
            averagescore_voted = 0
            testrandom = [0, 0, 0]
            for datas in tqdm(dataloader_test):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                
                targets_new = [
                    # 0 = neither hate speech or offensive language
                    # 1 = either hate speech or offensive language
                    torch.LongTensor([[0] if t == 2 else [1] for t in target]).to(device),
                    
                    # 0 = either offensive language or neither
                    # 1 = hate speech
                    torch.LongTensor([[1] if t == 0 else [0] for t in target]).to(device),
                    
                    # 0 = either hate speech or neither
                    # 1 = offensive language
                    torch.LongTensor([[1] if t == 1 else [0] for t in target]).to(device)
                ]
                
                preds = []
                
                for mi in range(len(models1)):
                    models1[mi].eval()
                    pred = models1[mi](list(tweet), has_mask)
                    preds.append(pred[0].argmax(-1).cpu().numpy())
                    loss = lossfns1[mi](pred[0], targets_new[mi].squeeze(1))
                    f1score = sklearn.metrics.f1_score(targets_new[mi].squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                    averageloss[mi] += loss.item()/len(dataloader_test)
                    averagescore[mi] += f1score/len(dataloader_test)
                
                preds_voted = np.empty(preds[0].shape[0])
                for pi in range(len(preds[0])):
                    hatespeech = 0
                    offensivelanguage = 0
                    neither = 0

                    # 0 = neither hate speech or offensive language
                    # 1 = either hate speech or offensive language
                    if preds[0][pi] == 0:
                        neither +=1
                    elif preds[0][pi] == 1:
                        hatespeech += 1
                        offensivelanguage += 1

                    # 0 = either offensive language or neither
                    # 1 = hate speech
                    if preds[1][pi] == 0:
#                         pass
                        offensivelanguage += 1
                        neither += 1
                    elif preds[1][pi] == 1:
                        hatespeech += 1

                    # 0 = either hate speech or neither
                    # 1 = offensive language
                    if preds[2][pi] == 0:
#                         pass
                        hatespeech += 1
                        neither += 1
                    elif preds[2][pi] == 1:
                        offensivelanguage += 1                

                    maxvotes = max(hatespeech, offensivelanguage, neither)
                    maxvotes_classes_count = 0
                    maxvotes_classes_indexes = []
                    if hatespeech == maxvotes:
                        maxvotes_classes_count += 1
                        maxvotes_classes_indexes.append(0)
                    if offensivelanguage == maxvotes:
                        maxvotes_classes_count += 1
                        maxvotes_classes_indexes.append(1)
                    if neither == maxvotes:
                        maxvotes_classes_count += 1
                        maxvotes_classes_indexes.append(2)

                    preds_voted[pi] = random.choice(maxvotes_classes_indexes)
                
                    if len(maxvotes_classes_indexes) == 1:
                        testrandom[0] += 1
                    elif len(maxvotes_classes_indexes) == 2:
                        testrandom[1] += 1
                    elif len(maxvotes_classes_indexes) == 3:
                        testrandom[2] += 1
                
                f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), preds_voted, average = 'weighted')
                averagescore_voted += f1score/len(dataloader_test)
                
            testloss.append(averageloss)
            testscore.append(averagescore)
            testscore_voted.append(f1score)
            
            
        print(f"epoch: {i+1}")
        print(f"train loss: {trainloss[-1]}\ntrain f1score: {trainscore[-1]}\ntrain f1score - voted: {trainscore_voted[-1]} (majority classes: 1 = {trainrandom[0]}, 2 = {trainrandom[1]}, 3 = {trainrandom[2]})")
        if dataloader_valid is not None:
            print(f"\nvalidation loss: {validloss[-1]}\nvalidation f1score: {validscore[-1]}\nvalidation f1score - voted: {validscore_voted[-1]} (majority classes: 1 = {validrandom[0]}, 2 = {validrandom[1]}, 3 = {validrandom[2]})")
        if dataloader_test is not None:
            print(f"\ntest loss: {testloss[-1]}\ntest f1score: {testscore[-1]}\ntest f1score - voted: {testscore_voted[-1]} (majority classes: 1 = {testrandom[0]}, 2 = {testrandom[1]}, 3 = {testrandom[2]})")

    return trainloss, validloss, testloss, trainscore, validscore, testscore, trainscore_voted, validscore_voted, testscore_voted

In [26]:
epochs1 = 20
trainloss1, validloss1, testloss1, trainscore1, validscore1, testscore1, trainscore1_voted, validscore1_voted, testscore1_voted = train1(
    dataloader_train1, dataloader_valid1, dataloader_test1,
    models1 = models1, optimizers1 = optimizers1, lossfns1 = lossfns1,
    epochs = epochs1, has_mask = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:39<00:00,  1.95it/s]


epoch: 1
train loss: [0.15081716323376543, 0.1773887502870733, 0.2538461972809126]
train f1score: [0.933827579634242, 0.9224440480904782, 0.8888148681803758]
train f1score - voted: 0.8650432413219666 (majority classes: 1 = 18898, 2 = 257, 3 = 671)

validation loss: [0.10451622706097671, 0.16335075062054857, 0.21466691548434583]
validation f1score: [0.9612810574997169, 0.937473785445638, 0.9141305170377739]
validation f1score - voted: 0.9014493648275438 (majority classes: 1 = 2370, 2 = 80, 3 = 29)

test loss: [0.10922672019268459, 0.16630997255635566, 0.2328551201484143]
test f1score: [0.9591479787350994, 0.9252821937897402, 0.906145703027181]
test f1score - voted: 0.9319727891156463 (majority classes: 1 = 2357, 2 = 81, 3 = 40)


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:40<00:00,  1.92it/s]


epoch: 2
train loss: [0.10142584374219545, 0.1520448130585493, 0.20118602057618493]
train f1score: [0.9611705230425094, 0.9300327228750007, 0.9193426589772313]
train f1score - voted: 0.8973639616739996 (majority classes: 1 = 19004, 2 = 317, 3 = 505)

validation loss: [0.10456840839619057, 0.1531045924490079, 0.2136974470833173]
validation f1score: [0.9605324726623826, 0.9399880044940739, 0.9110708751738215]
validation f1score - voted: 0.9020027595021185 (majority classes: 1 = 2377, 2 = 35, 3 = 67)

test loss: [0.1081122840897968, 0.16221164343639818, 0.24035571744808784]
test f1score: [0.9573858335433473, 0.929863682734856, 0.9009670737071558]
test f1score - voted: 0.9319727891156463 (majority classes: 1 = 2393, 2 = 35, 3 = 50)


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:37<00:00,  2.07it/s]


...

epoch: 18
train loss: [0.03760405292757038, 0.07124866449552023, 0.09654953527760003]
train f1score: [0.9852942363372132, 0.9694836501634635, 0.9601827056339706]
train f1score - voted: 0.953799228284632 (majority classes: 1 = 18748, 2 = 508, 3 = 570)

validation loss: [0.17048272838329487, 0.21843676852325022, 0.3839040073470618]
validation f1score: [0.9450849268263811, 0.9277506928855657, 0.8998912983259414]
validation f1score - voted: 0.883922135011572 (majority classes: 1 = 2304, 2 = 72, 3 = 103)

test loss: [0.1694448267653337, 0.23961405578451475, 0.4168713602285165]
test f1score: [0.9498780279588257, 0.9264894487435593, 0.895853320581514]
test f1score - voted: 0.9319727891156463 (majority classes: 1 = 2312, 2 = 69, 3 = 97)


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.41it/s]


epoch: 19
train loss: [0.03576253206622146, 0.06825356322703056, 0.09574693458703082]
train f1score: [0.985092447576611, 0.9707051934406936, 0.9590448455723688]
train f1score - voted: 0.9504927195271814 (majority classes: 1 = 18697, 2 = 528, 3 = 601)

validation loss: [0.1446706109877246, 0.23265817973953784, 0.3278838435952098]
validation f1score: [0.9601077761839063, 0.9304226668313322, 0.8961409837378862]
validation f1score - voted: 0.8862158845564604 (majority classes: 1 = 2266, 2 = 126, 3 = 87)

test loss: [0.15365098080991837, 0.24597579332737202, 0.3442973932490135]
test f1score: [0.9620620361867351, 0.9287142026799863, 0.8966613153581832]
test f1score - voted: 0.8163265306122449 (majority classes: 1 = 2297, 2 = 105, 3 = 76)


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:22<00:00,  3.39it/s]

epoch: 20
train loss: [0.03265342246083873, 0.0680925662169278, 0.09290072149221573]
train f1score: [0.9869855713188511, 0.9715565048275652, 0.9606349553248982]
train f1score - voted: 0.9533142502835056 (majority classes: 1 = 18743, 2 = 529, 3 = 554)

validation loss: [0.16960317592878332, 0.2299722771948346, 0.320791245605319]
validation f1score: [0.9581905890511946, 0.9285938754832407, 0.8990042140869718]
validation f1score - voted: 0.8891346246493117 (majority classes: 1 = 2276, 2 = 112, 3 = 91)

test loss: [0.17315609503062804, 0.2520025126659909, 0.33756032278044856]
test f1score: [0.9601339395712746, 0.9212530907578803, 0.8982509302227984]
test f1score - voted: 0.8571428571428571 (majority classes: 1 = 2305, 2 = 87, 3 = 86)


In [27]:
trainloss1

[[0.15081716323376543, 0.1773887502870733, 0.2538461972809126],
 [0.10142584374219545, 0.1520448130585493, 0.20118602057618493],
 [0.09215844936496129, 0.14430199827878704, 0.19469716437761825],
 [0.0841458622069518, 0.13809858911610648, 0.1857498210854827],
 [0.07882496096764596, 0.13140060677223153, 0.17704597890917806],
 [0.07261776992142381, 0.12493172170624396, 0.16952790591505268],
 [0.06592454281053509, 0.11797809376110958, 0.16165151310063194],
 [0.060644900391156144, 0.11019304664505097, 0.15421202497166245],
 [0.05657680004951548, 0.1051254910600912, 0.14919385268923743],
 [0.05143115481966351, 0.09866545810364194, 0.1421510172452055],
 [0.051309625119076, 0.09297930252618124, 0.13291206500162517],
 [0.04628356534080772, 0.08969270409790861, 0.12652920452772737],
 [0.04355954034131911, 0.0850836672549767, 0.12248894228945452],
 [0.04356910845693371, 0.08242754256953641, 0.11561073204277153],
 [0.040219423077313306, 0.07705399796813574, 0.11132601533956347],
 [0.03805548823785

In [28]:
validloss1

[[0.10451622706097671, 0.16335075062054857, 0.21466691548434583],
 [0.10456840839619057, 0.1531045924490079, 0.2136974470833173],
 [0.10672798891289115, 0.1523498896127328, 0.20836718891484615],
 [0.10570904647167294, 0.15272362112330312, 0.21461172912938478],
 [0.11131164849473116, 0.15382342379635727, 0.22012268455746845],
 [0.11280818104457391, 0.16097111235826447, 0.22666168189010566],
 [0.1063075586914634, 0.16135736530981004, 0.23566299333022192],
 [0.11652457494384201, 0.17302918080718088, 0.23185923537955835],
 [0.1264461581953443, 0.17632747732867046, 0.2449703800659149],
 [0.1324986708828081, 0.19041891665890426, 0.24468466682502857],
 [0.14069902344057578, 0.18142238794228968, 0.2591885552287866],
 [0.14739312987494027, 0.18196655668986914, 0.26434449708232505],
 [0.1363735561817287, 0.21609120324660003, 0.32829313436284285],
 [0.1611589886134682, 0.17941129090598765, 0.28492663571467763],
 [0.15843129373462994, 0.2167957392342102, 0.305519612147831],
 [0.137928287504623, 0.

In [29]:
testloss1

[[0.10922672019268459, 0.16630997255635566, 0.2328551201484143],
 [0.1081122840897968, 0.16221164343639818, 0.24035571744808784],
 [0.1078942482216427, 0.15822246584754726, 0.232594211323139],
 [0.11392749032865349, 0.15981840121392646, 0.24493077948976014],
 [0.11483832456780457, 0.15589847752394584, 0.23937807912723386],
 [0.1172983975519832, 0.1638180702590407, 0.2400284215139273],
 [0.1148274310816748, 0.16672303575353742, 0.2599486108773794],
 [0.12141902000905995, 0.172109608251888, 0.24531008073916796],
 [0.13196149949605268, 0.17813003819005993, 0.25982233366140955],
 [0.1246568262905408, 0.18614913222308346, 0.2563116139230821],
 [0.1397761303307608, 0.18856869019472447, 0.2871797337459447],
 [0.1482739842292339, 0.19759127352004632, 0.28485957867442047],
 [0.13506640417453572, 0.23336364341995275, 0.34589006326710575],
 [0.15128903821609224, 0.1867227251760853, 0.29480186166862654],
 [0.1500688168020226, 0.23752484391503126, 0.32412141733444655],
 [0.14048562102760073, 0.2159

In [30]:
trainscore1

[[0.933827579634242, 0.9224440480904782, 0.8888148681803758],
 [0.9611705230425094, 0.9300327228750007, 0.9193426589772313],
 [0.9651214568364537, 0.9353536279807713, 0.9228222434224489],
 [0.9690322831218746, 0.9401456979626658, 0.9242142539957554],
 [0.9719964307718245, 0.9423627461204164, 0.9292700369654386],
 [0.9727842654944581, 0.9444649962421602, 0.9320185591556597],
 [0.9744881283359149, 0.9478759926236542, 0.9333691649687905],
 [0.9783085817540592, 0.9521591439246363, 0.9365404478722955],
 [0.978258683974075, 0.9558985491528422, 0.9363815307354281],
 [0.9799200789346469, 0.9557735797265072, 0.9408491030056383],
 [0.9804741071225946, 0.9598794650993289, 0.9435017673404388],
 [0.9825426168716815, 0.9616209952559236, 0.9464762209962798],
 [0.9831418073097289, 0.9648007808361506, 0.9480667824897427],
 [0.9833027851357327, 0.9654148572636928, 0.9489554580052855],
 [0.9844155105282281, 0.9673909342438847, 0.952438174772634],
 [0.9842104537986671, 0.9680662003438242, 0.95447798324478

In [31]:
validscore1

[[0.9612810574997169, 0.937473785445638, 0.9141305170377739],
 [0.9605324726623826, 0.9399880044940739, 0.9110708751738215],
 [0.9574553662349838, 0.9346799223494444, 0.9189615020289912],
 [0.9604176747758209, 0.9390214212988057, 0.9171356259542222],
 [0.9609143091937129, 0.9352008927692921, 0.9161390948765087],
 [0.9607503438144143, 0.9404950764487998, 0.9129448257548315],
 [0.957383960782972, 0.9372014351624214, 0.9089252104781894],
 [0.9572031072769411, 0.9329763682760923, 0.9099113584777384],
 [0.9606005450797854, 0.9355233888314711, 0.9095969673731206],
 [0.9542622880809324, 0.9301745459796306, 0.9115848389346672],
 [0.9619588997479944, 0.92960528013425, 0.9031821570438544],
 [0.9560687553412734, 0.9292880501937097, 0.899183340019521],
 [0.9501535375525313, 0.92945322929273, 0.8918074475500481],
 [0.9444713397558535, 0.9352796832682552, 0.8938583427320352],
 [0.9574911754584609, 0.9305121867653474, 0.9087210982377825],
 [0.9529824214248755, 0.9333805460154065, 0.90405670761054],
 

In [32]:
testscore1

[[0.9591479787350994, 0.9252821937897402, 0.906145703027181],
 [0.9573858335433473, 0.929863682734856, 0.9009670737071558],
 [0.9552495601246498, 0.9282575277499822, 0.9062052329099028],
 [0.9572990191216727, 0.9274626238650794, 0.9077656993122714],
 [0.9577254809819364, 0.9265857414239637, 0.9025718137740812],
 [0.9570735817739567, 0.929685429005297, 0.9014459053494963],
 [0.9564683544757375, 0.9259003261817145, 0.905633144697302],
 [0.9600056239389458, 0.9285928614210686, 0.9022544144559225],
 [0.9571944338932, 0.927239944120497, 0.9016513377907196],
 [0.9601703445022887, 0.93260845009264, 0.9013803774137866],
 [0.9599139789590548, 0.9270393694473148, 0.9041505747958528],
 [0.9577028132791919, 0.9265979628143903, 0.8995491885548608],
 [0.9591396072190793, 0.9251165301861921, 0.8847348762122933],
 [0.9558412580852688, 0.928958632400504, 0.8938051387761357],
 [0.9622448176867813, 0.9267492855993087, 0.908325109840028],
 [0.957692325549904, 0.92545107727961, 0.9008705543582425],
 [0.961

In [33]:
trainscore1_voted

[0.8650432413219666,
 0.8973639616739996,
 0.9033486791400858,
 0.9085794004336202,
 0.9158129006503266,
 0.9185656548005162,
 0.9206947256521137,
 0.9264417914349936,
 0.9284305134147315,
 0.931987202490466,
 0.9362752721373977,
 0.9394084636620548,
 0.9410672882671878,
 0.9440149990568821,
 0.9478470503893477,
 0.948716759743865,
 0.9480606246692186,
 0.953799228284632,
 0.9504927195271814,
 0.9533142502835056]

In [34]:
validscore1_voted

[0.9014493648275438,
 0.9020027595021185,
 0.8963636531896864,
 0.9015321792505036,
 0.8996228963452921,
 0.9029549324575761,
 0.8953651616708633,
 0.8958350173115998,
 0.9012766054846975,
 0.8902958030266244,
 0.8923140215420106,
 0.8865252582480907,
 0.8778218924895496,
 0.8797578701934788,
 0.8935568537905328,
 0.8929703902105415,
 0.89033579581728,
 0.883922135011572,
 0.8862158845564604,
 0.8891346246493117]

In [35]:
testscore1_voted

[0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.9319727891156463,
 0.8571428571428571,
 0.9319727891156463,
 0.8571428571428571,
 0.9319727891156463,
 0.9319727891156463,
 0.8571428571428571,
 0.9319727891156463,
 0.8163265306122449,
 0.8571428571428571]